In [1]:
import os


import instaloader
import yadisk

from decouple import config
from tqdm.notebook import tqdm
from tqdm import trange

In [6]:
L = instaloader.Instaloader(save_metadata=False, 
                            download_comments=False, 
                            download_geotags=False, 
                            download_videos=False, 
                            filename_pattern="{date_utc}_UTC_profile_{profile}",
                            quiet=True)

L.login(config('U'), config('P'))
hashtag = instaloader.Hashtag.from_name(L.context, 'tattoosketches')

In [7]:
API_KEY = config('KEY')
y = yadisk.YaDisk(token=API_KEY)

assert y.check_token()

`profile.get_posts()` is a NodeIterator instance and supports state freezing. `hashtag.get_posts()` is a python generator and does not support freezing. :(

In [8]:
def save_photos_batch(post_iterable, n_photos, save_dir_name):
    for i, post in tqdm(enumerate(post_iterable), 
                        desc='instagram images scrapping bar', 
                        total=n_photos,
                        leave=False):
        if i >= n_photos:
            break
        try:
            L.download_post(post, target=save_dir_name)
        except instaloader.exceptions.InstaloaderException as exc:
            print(f'post download failed with {exc}')

In [9]:
def upload_files_from_dir(src_path, dst_path, y):
    try:
        y.mkdir(dst_path)
    except yadisk.exceptions.DirectoryExistsError:
        pass
    for filename in tqdm(os.listdir(src_path), desc='uploading images to y.disk', leave=False):
        try:
            y.upload(os.path.join(src_path, filename), 
                     os.path.join(dst_path, filename))
        except yadisk.exceptions.PathExistsError:
            pass

In [10]:
def clear_dir(dir_path):
    for filename in os.listdir(dir_path):
        os.remove(os.path.join(dir_path, filename))

In [11]:
hashtag = instaloader.Hashtag.from_name(L.context, 'tattoosketches')

In [12]:
hashtag_iterable = hashtag.get_all_posts()

In [13]:
dir_name = 'tattoosketches_all'

for _ in tqdm(range(1000)):
    save_photos_batch(hashtag_iterable, 20, dir_name)
    upload_files_from_dir(dir_name, 'app:/insta/' + dir_name, y)
    clear_dir(dir_name)

JSON Query to api/v1/media/2501805042635681042/info/: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    1    1    3    3    3    3
   2b0673e0dc4580674a88d426fe00ea90:   81   93  115  115  115  115
 *                           iphone:   79   90  110  110  110  110
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, do not use any Instagram App while Instaloader
is running.
The request will be retried in 13 minutes, at 15:00.
JSON Query to api/v1/media/2500881005335741160/info/: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    0    0    1    1    1    3
   2b0673e0dc4580674a88d426fe00ea90:    1    1   76   76  111  116
 *                           iphone:    2   

KeyboardInterrupt: 